# RDF und SPARQL Sandkasten

In [ ]:
# nötige Module importieren
%pip install -q ipywidgets==8.1.1 ipycytoscape networkx rdflib
import rdflib
import ipycytoscape as cy
import networkx as nx
import pandas as pd

## Funktionsdefinitionen

In [ ]:
# Funktionsfür die Konvertierung von RDF zu NetworkX
def rdflib_to_networkx(rdf_graph):
    G = nx.DiGraph()  # Use Graph() if you want an undirected graph
    for subj, pred, obj in rdf_graph:
        if isinstance(subj, rdflib.URIRef):
            subj = rdf_graph.qname(subj)
        elif isinstance(subj, rdflib.BNode):
            subj = f"_:bnode_{subj}"  # Convert blank node to a unique string
        pred = rdf_graph.qname(pred)
        if isinstance(obj, rdflib.URIRef):
            obj = rdf_graph.qname(obj)
        elif isinstance(obj, rdflib.BNode):
            obj = f"_:bnode_{obj}"  # Convert blank node to a unique string
        elif isinstance(obj, rdflib.Literal):
            obj = str(obj)  # Convert Literal to its string representation
        G.add_edge(str(subj), str(obj), label=str(pred))
    return G

In [ ]:
# Funktionsdefinition, um einen TTL String zu parsen und als Graph zu plotten
def parse_and_plot(ttl_string):
    g = rdflib.Graph()
    g.parse(data = ttl_string)
    nx_graph = rdflib_to_networkx(g)
    plot = cy.CytoscapeWidget()
    plot.graph.add_graph_from_networkx(nx_graph, directed=True)

    style = [
        {
            'selector': 'node',
             'style': {
                'font-family': 'helvetica',
                'font-size': '12px',
                 'color': 'white',
                'text-outline-width': 2,
                'text-outline-color': 'green',
                'background-color': 'green',
                'content': 'data(id)',
                'text-valign': 'center',
             }
        },
        {
            'selector': 'edge.directed',
            'style': {
                'font-family': 'helvetica',
                'font-size': '12px',
                'label': 'data(label)',
                'color': 'white',
                'text-outline-width': 2,
                'text-outline-color': 'orange',
                'background-color': 'orange',
                'curve-style': 'bezier',
                'target-arrow-shape': 'triangle',
            }
        }
    ]
    
    plot.set_style(style)
    return plot

In [ ]:
# Funktionsdefinition, um eine Query gegen eine lokale Turtle Datei auszuführen
def query(ttl_string, query_string):
    
    g = rdflib.Graph()
    g.parse(data = ttl_string)

    qres = g.query(query_string)

    df = pd.DataFrame(qres, columns=qres.vars)
    return df

## Spezifikationen

Diese technischen Dokumente sind sehr lesenwert (und für Spezifikationen auch sehr leserlich geschrieben):

- [RDF Primer](https://www.w3.org/TR/rdf11-primer/)
- [Turtle](https://www.w3.org/TR/turtle/)
- [SPARQL](https://www.w3.org/TR/sparql11-query/)

## Basis Turtle

Bilde folgende Informationen als RDF Triple in Turtle Serialisierung ab:

- Karin Keller Sutter ist eine FDP Bundesrätin
- Viola Amherd ist eine Bundesrätin in der Partei "die Mitte"
- NadineMasshardt ist eine SP Nationalrätin, die in Affoltern am Albis geboren wurde
- Maya Graf ist eine Ständerätin der Grünen und ihr Vater ist Fritz Graf
- Fritz Graf ist im Baselbieter Landrat für die SVP und Maya Graf seine Tochter
- FDP, die Mitte, SP, Grüne und SVP sind politische Parteien 

Nutze für die URIs `<https://example.com/...>`

In [ ]:
ttl_string = """

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit Prefix

Ersetze in Deinen Triples `<https://example.com/>` durch einen Prefix (am besten den Empty Prefix; [Base vs. Empty Prefix](https://stackoverflow.com/questions/34146707/turtle-difference-between-base-and-empty-prefix))

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL ohne Wiederholungen

Ersetze doppelte Subjekte mit Hilfe von [Prädikat-Listen](https://www.w3.org/TR/turtle/#predicate-lists).

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit echter URI

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit externem Vokabular


In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit Literals

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit Blank Nodes

In [ ]:
ttl_string = """

@prefix : <https://ld.di.digisus-lab.ch/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## SPARQL Base Query

In [ ]:
query_string = """

PREFIX : <https://ld.di.digisus-lab.ch/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?s ?p ?o.

}

"""

df = query(ttl_string, query_string)
display(df)